# Data Preperation GPS
Author: P.C.O. Maseland <br>
Date: 03 December 2021<br>

This ipynb file contains script to merge all seperate csv files into one whole dataset of GPS.

In [1]:
#Import packages/libraries
import os
import pandas as pd
import csv
from zipfile import ZipFile
import numpy as np

#### Directory / file handling

In [43]:
#opening data files
path = 'Rawdata2/'
dirs = os.listdir(path)

In [44]:
#find folders
for file in dirs:
    print(file)

proband1
proband10
proband11
proband12
proband13
proband14
proband15
proband2
proband3
proband4
proband5
proband6
proband7
proband8
proband9


In [45]:
#create a list with all zip file paths
subpath = ''
count = 0
paths_list = []

for proband in dirs:
    subpath = path + proband
    subdir = os.listdir(subpath)
    for foldername in subdir:
        if foldername == 'data':
            subsubpath = subpath + "/data"
            subsubdir = os.listdir(subsubpath)
            for file in subsubdir:
                newpath = subsubpath + "/" + file
                paths_list.append(newpath)
                count += 1
        else:
            break

In [46]:
#all zip file paths
paths_list

['Rawdata2/proband1/data/acc_climbingdown_csv.zip',
 'Rawdata2/proband1/data/acc_climbingdown_sqlite.zip',
 'Rawdata2/proband1/data/acc_climbingup_csv.zip',
 'Rawdata2/proband1/data/acc_climbingup_sqlite.zip',
 'Rawdata2/proband1/data/acc_jumping_csv.zip',
 'Rawdata2/proband1/data/acc_jumping_sqlite.zip',
 'Rawdata2/proband1/data/acc_lying_csv.zip',
 'Rawdata2/proband1/data/acc_lying_sqlite.zip',
 'Rawdata2/proband1/data/acc_running_csv.zip',
 'Rawdata2/proband1/data/acc_running_sqlite.zip',
 'Rawdata2/proband1/data/acc_sitting_csv.zip',
 'Rawdata2/proband1/data/acc_sitting_sqlite.zip',
 'Rawdata2/proband1/data/acc_standing_csv.zip',
 'Rawdata2/proband1/data/acc_standing_sqlite.zip',
 'Rawdata2/proband1/data/acc_walking_csv.zip',
 'Rawdata2/proband1/data/acc_walking_sqlite.zip',
 'Rawdata2/proband1/data/gps_climbingdown_csv.zip',
 'Rawdata2/proband1/data/gps_climbingdown_sqlite.zip',
 'Rawdata2/proband1/data/gps_climbingup_csv.zip',
 'Rawdata2/proband1/data/gps_climbingup_sqlite.zip',


In [47]:
#remove sqlite.zip files and only use GPS.zip files
paths_list = [x for x in paths_list if "sqlite" not in x and "gps" in x]
paths_list

['Rawdata2/proband1/data/gps_climbingdown_csv.zip',
 'Rawdata2/proband1/data/gps_climbingup_csv.zip',
 'Rawdata2/proband1/data/gps_jumping_csv.zip',
 'Rawdata2/proband1/data/gps_lying_csv.zip',
 'Rawdata2/proband1/data/gps_running_csv.zip',
 'Rawdata2/proband1/data/gps_sitting_csv.zip',
 'Rawdata2/proband1/data/gps_standing_csv.zip',
 'Rawdata2/proband1/data/gps_walking_csv.zip',
 'Rawdata2/proband10/data/gps_climbingdown_csv.zip',
 'Rawdata2/proband10/data/gps_climbingup_csv.zip',
 'Rawdata2/proband10/data/gps_jumping_csv.zip',
 'Rawdata2/proband10/data/gps_lying_csv.zip',
 'Rawdata2/proband10/data/gps_running_csv.zip',
 'Rawdata2/proband10/data/gps_sitting_csv.zip',
 'Rawdata2/proband10/data/gps_standing_csv.zip',
 'Rawdata2/proband10/data/gps_walking_csv.zip',
 'Rawdata2/proband11/data/gps_climbingdown_csv.zip',
 'Rawdata2/proband11/data/gps_climbingup_csv.zip',
 'Rawdata2/proband11/data/gps_jumping_csv.zip',
 'Rawdata2/proband11/data/gps_lying_csv.zip',
 'Rawdata2/proband11/data/gp

#### Zipfile handling
function 'openzip' has as input a zipfile path and SubjectNr. The function extract all csv files from zip, removes the readMe text file and retrieves info from file name:
* Sensory_type
* Activity
* Body-position

As output the function generates a list with csv file's and list of labels that stick with that csv file.

In [48]:
#input zipfilename, output list of csv files
def openzip(file_name, subject_id):
    labellist = []
    
    with ZipFile(file_name, 'r') as zip:
        csvlist = zip.namelist()
    
        if 'readMe' in csvlist:
            csvlist.remove('readMe')

        for name in csvlist:
            sublabellist = []
            zip.extract(name)

            #get labels
            templist = name.split("_")
            sensory_type = templist[0]
            activity = templist[1]
            body_position = templist[2].split(".")
            body_position = body_position[0]

            #create sublist in csvlist per csv file for labels
            sublabellist.append(sensory_type)
            sublabellist.append(activity)
            sublabellist.append(body_position)

            labellist.append(sublabellist)

        return csvlist, labellist

The 'Create_dataframe' function has as input the csvfile name, the according labellist and a count the nr of dataframe that is created. Since we merging per sensor-type, activity and subject the attributes. Meaning that we only need the Subject_ID, sensor-type and activty column 1 time. <br>

Further, the column attributes name will be changed according the body-position the data is from

With as output a dataframe.

In [49]:
def create_dataframe(csvfile, labellist, count):
    df = pd.read_csv(csv)
    
    if count == 0:
        #add columns to df using labellist
        df['Subject ID'] = subject_ID
        df['Sensor-type'] = labellist[count][0]
        df['Activity'] = labellist[count][1]
          
    body_position = labellist[count][2]  
        
    df = df.rename(columns=
                {'attr_lat': 'attr_lat_' + body_position,
                 'attr_lng': 'attr_lng_' + body_position})
    return df

The main code path down below, retrieves for each zip file from which subject it is and generates a csv file containing the merged dataframe of each sensor-type, activity, subject combination. 

In [50]:
for path in paths_list:
    #retrieve labels
    templist = path.split("/")
    templist2 = templist[3].split("_")

    #get subject label
    subject_ID = templist[1].replace("proband", '')
    
    print("PATH: " + path + " SUBJECT: " + subject_ID)
    
    count = 0
    csvlist, labellist = openzip(path, subject_ID)
    
    for csv in csvlist:
        
        if count == 0:
            #first_df
            df1 = create_dataframe(csv, labellist, count)
            #print('START dataframe    ' + csv)
        else:
            #merge
            df2 = create_dataframe(csv, labellist, count)
            merged_df = df1.merge(df2, on='id')
            df1 = merged_df
            #print('MERGE COMPLETE    ' + csv)
        
        count += 1
        
        #remove csv after use
        os.remove(csv)  
        
    #temporary csv
    filename = subject_ID + '_' + labellist[0][0] + '_' + labellist[0][1] + '_' + 'Merged.csv'
    print("MERGE FILE COMPLETE: " + filename)
    
    merged_df.to_csv(filename, index=True)
        
        

PATH: Rawdata2/proband1/data/gps_climbingdown_csv.zip SUBJECT: 1
MERGE FILE COMPLETE: 1_GPS_climbingdown_Merged.csv
PATH: Rawdata2/proband1/data/gps_climbingup_csv.zip SUBJECT: 1
MERGE FILE COMPLETE: 1_GPS_climbingup_Merged.csv
PATH: Rawdata2/proband1/data/gps_jumping_csv.zip SUBJECT: 1
MERGE FILE COMPLETE: 1_GPS_jumping_Merged.csv
PATH: Rawdata2/proband1/data/gps_lying_csv.zip SUBJECT: 1
MERGE FILE COMPLETE: 1_GPS_lying_Merged.csv
PATH: Rawdata2/proband1/data/gps_running_csv.zip SUBJECT: 1
MERGE FILE COMPLETE: 1_GPS_running_Merged.csv
PATH: Rawdata2/proband1/data/gps_sitting_csv.zip SUBJECT: 1
MERGE FILE COMPLETE: 1_GPS_sitting_Merged.csv
PATH: Rawdata2/proband1/data/gps_standing_csv.zip SUBJECT: 1
MERGE FILE COMPLETE: 1_GPS_standing_Merged.csv
PATH: Rawdata2/proband1/data/gps_walking_csv.zip SUBJECT: 1
MERGE FILE COMPLETE: 1_GPS_walking_Merged.csv
PATH: Rawdata2/proband10/data/gps_climbingdown_csv.zip SUBJECT: 10
MERGE FILE COMPLETE: 10_GPS_climbingdown_Merged.csv
PATH: Rawdata2/prob

PATH: Rawdata2/proband4/data/gps_standing_csv.zip SUBJECT: 4
MERGE FILE COMPLETE: 4_GPS_standing_Merged.csv
PATH: Rawdata2/proband4/data/gps_walking_csv.zip SUBJECT: 4
MERGE FILE COMPLETE: 4_GPS_walking_Merged.csv
PATH: Rawdata2/proband5/data/gps_climbingdown_csv.zip SUBJECT: 5
MERGE FILE COMPLETE: 5_GPS_climbingdown_Merged.csv
PATH: Rawdata2/proband5/data/gps_climbingup_csv.zip SUBJECT: 5
MERGE FILE COMPLETE: 5_GPS_climbingup_Merged.csv
PATH: Rawdata2/proband5/data/gps_jumping_csv.zip SUBJECT: 5
MERGE FILE COMPLETE: 5_GPS_jumping_Merged.csv
PATH: Rawdata2/proband5/data/gps_lying_csv.zip SUBJECT: 5
MERGE FILE COMPLETE: 5_GPS_lying_Merged.csv
PATH: Rawdata2/proband5/data/gps_running_csv.zip SUBJECT: 5
MERGE FILE COMPLETE: 5_GPS_running_Merged.csv
PATH: Rawdata2/proband5/data/gps_sitting_csv.zip SUBJECT: 5
MERGE FILE COMPLETE: 5_GPS_sitting_Merged.csv
PATH: Rawdata2/proband5/data/gps_standing_csv.zip SUBJECT: 5
MERGE FILE COMPLETE: 5_GPS_standing_Merged.csv
PATH: Rawdata2/proband5/data/g

In [51]:
#adjust to what sensor you want a file to merge
#make dataset containing all csv's of accuracy
import glob
dir = os.getcwd()
dirs = os.listdir(dir)
sensor = 'GPS'

for item in dirs:
    newdirs = glob.glob('*' + sensor + '*.csv') 
            

In [52]:
count = 0
for csv in newdirs:    
    if count == 0:
        df = pd.read_csv(csv)
    else: 
        df2 = pd.read_csv(csv)
        merged_df = df.append(df2)
        df = merged_df
        print(csv)
        print(merged_df.shape)
    count += 1

10_GPS_climbingup_Merged.csv
(57, 23)
10_GPS_jumping_Merged.csv
(63, 23)
10_GPS_lying_Merged.csv
(122, 23)
10_GPS_running_Merged.csv
(166, 23)
10_GPS_sitting_Merged.csv
(199, 23)
10_GPS_standing_Merged.csv
(225, 23)
10_GPS_walking_Merged.csv
(270, 23)
11_GPS_climbingdown_Merged.csv
(308, 23)
11_GPS_climbingup_Merged.csv
(344, 23)
11_GPS_jumping_Merged.csv
(352, 23)
11_GPS_lying_Merged.csv
(384, 23)
11_GPS_running_Merged.csv
(427, 23)
11_GPS_sitting_Merged.csv
(458, 23)
11_GPS_standing_Merged.csv
(486, 23)
11_GPS_walking_Merged.csv
(542, 23)
12_GPS_climbingdown_Merged.csv
(579, 23)
12_GPS_climbingup_Merged.csv
(611, 23)
12_GPS_jumping_Merged.csv
(620, 23)
12_GPS_lying_Merged.csv
(651, 23)
12_GPS_running_Merged.csv
(674, 23)
12_GPS_sitting_Merged.csv
(713, 23)
12_GPS_standing_Merged.csv
(749, 23)
12_GPS_walking_Merged.csv
(767, 23)
13_GPS_climbingdown_Merged.csv
(791, 23)
13_GPS_climbingup_Merged.csv
(828, 23)
13_GPS_jumping_Merged.csv
(833, 23)
13_GPS_lying_Merged.csv
(865, 23)
13_GPS_r

In [55]:
merged_df.head(100)

,Unnamed: 0,id,attr_time_x,attr_lat_chest,attr_lng_chest,Subject ID,Sensor-type,Activity,attr_time_y,attr_lat_head,...,attr_time_y.1,attr_lat_thigh,attr_lng_thigh,attr_time_x.2,attr_lat_upperarm,attr_lng_upperarm,attr_time_y.2,attr_lat_waist,attr_lng_waist,attr_time
0,0,1,1436802577000,49.449901,8.898195,10,GPS,climbingdown,1436802579000,49.449862,...,1.436803e+12,49.449874,8.898413,1.436803e+12,49.450039,8.898408,1.436803e+12,49.449828,8.898438,NaN
1,1,2,1436802586000,49.449884,8.898237,10,GPS,climbingdown,1436802591000,49.449861,...,1.436803e+12,49.449874,8.898411,1.436803e+12,49.450039,8.898411,1.436803e+12,49.449836,8.898439,NaN
2,2,3,1436802611000,49.449846,8.898393,10,GPS,climbingdown,1436802610000,49.449888,...,1.436803e+12,49.449874,8.898411,1.436803e+12,49.450052,8.898427,1.436803e+12,49.449847,8.898437,NaN
3,3,4,1436802665000,49.450071,8.898063,10,GPS,climbingdown,1436802621000,49.449914,...,1.436803e+12,49.449884,8.898429,1.436803e+12,49.450052,8.898421,1.436803e+12,49.449848,8.898437,NaN
4,4,5,1436802675000,49.450073,8.898262,10,GPS,climbingdown,1436802640000,49.450150,...,1.436803e+12,49.449885,8.898430,1.436803e+12,49.450051,8.898414,1.436803e+12,49.449852,8.898435,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32,32,33,1436803791000,49.448793,8.900309,10,GPS,lying,1436803786000,49.448775,...,1.436804e+12,49.448774,8.900388,1.436804e+12,49.448869,8.900397,1.436804e+12,49.448859,8.900212,NaN
33,33,34,1436803800000,49.448792,8.900309,10,GPS,lying,1436803796780,49.448899,...,1.436804e+12,49.448775,8.900388,1.436804e+12,49.448869,8.900396,1.436804e+12,49.448859,8.900214,NaN
34,34,35,1436803822000,49.448792,8.900307,10,GPS,lying,1436803817780,49.448670,...,1.436804e+12,49.448773,8.900386,1.436804e+12,49.448868,8.900400,1.436804e+12,49.448858,8.900216,NaN
35,35,36,1436803833000,49.448792,8.900306,10,GPS,lying,1436803829781,49.448761,...,1.436804e+12,49.448772,8.900386,1.436804e+12,49.448870,8.900395,1.436804e+12,49.448858,8.900219,NaN


In [59]:
merged_df.columns

Index(['id', 'attr_lat_chest', 'attr_lng_chest', 'Subject ID', 'Sensor-type',
       'Activity', 'attr_lat_head', 'attr_lng_head', 'attr_lat_shin',
       'attr_lng_shin', 'attr_lat_thigh', 'attr_lng_thigh',
       'attr_lat_upperarm', 'attr_lng_upperarm', 'attr_lat_waist',
       'attr_lng_waist'],
      dtype='object')

In [58]:
#Drop unneccesary columns
merged_df = merged_df.drop(columns=['Unnamed: 0',
                     'attr_time',
                     'attr_time_x',
                     'attr_time_y',
                     'attr_time_x.1',
                     'attr_time_y.1',
                     'attr_time_x.2',
                     'attr_time_y.2'])

In [60]:
merged_df.to_csv("tempfile_GPS.csv")

In [61]:
merged_df.shape

(3841, 16)

In [62]:
#look at NaN values
merged_df.isnull().sum(axis = 0)

id                    0
attr_lat_chest       87
attr_lng_chest       87
Subject ID            0
Sensor-type           0
Activity              0
attr_lat_head         0
attr_lng_head         0
attr_lat_shin        38
attr_lng_shin        38
attr_lat_thigh       36
attr_lng_thigh       36
attr_lat_upperarm    88
attr_lng_upperarm    88
attr_lat_waist       38
attr_lng_waist       38
dtype: int64